In [2]:
# Re-run with a robust extraction of coefficients for the mixed model.
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm, AnovaRM
from statsmodels.multivariate.manova import MANOVA

In [3]:
rng = np.random.default_rng(11)

In [4]:
def explain(title, question, structure, formula, interpretation_lines):
    print(f"\n{'='*80}\n{title}\n{'='*80}")
    print(f"Pregunta que responde:\n  - {question}")
    print("Estructura de los datos:\n" + "\n".join([f"  - {s}" for s in structure]))
    print(f"Fórmula del modelo:\n  {formula}")
    if interpretation_lines:
        print("Cómo interpretar (mini guía):")
        for line in interpretation_lines:
            print("  - " + line)

## Definición

### ¿Qué es un ANOVA de una vía?

El **ANOVA de una vía** (*one-way Analysis of Variance*) es una prueba estadística utilizada para comparar las **medias de una variable cuantitativa** entre **tres o más grupos independientes**, definidos por un **único factor categórico**.

Su objetivo es determinar si existen diferencias estadísticamente significativas entre las medias de los grupos, es decir, si todos los grupos provienen de la misma población o si al menos uno difiere.

---

### Estructura del modelo

El modelo general puede expresarse como:

[
Y_{ij} = \mu + \tau_i + \varepsilon_{ij}
]

donde:

* (Y_{ij}): valor observado de la variable dependiente en el grupo (i) y observación (j).
* (\mu): media general (promedio de todos los grupos).
* (\tau_i): efecto del grupo (i) (diferencia de la media del grupo respecto a la media general).
* (\varepsilon_{ij}): error aleatorio o variación individual, con (\varepsilon_{ij} \sim N(0, \sigma^2)).

---

### Hipótesis

[
H_0: \mu_A = \mu_B = \mu_C
]
[
H_1: \text{al menos una media difiere.}
]

---

### Supuestos

1. **Independencia** entre observaciones.
2. **Normalidad** de los residuos.
3. **Homogeneidad de varianzas** entre grupos (puede comprobarse con el test de Levene o Bartlett).

---

In [7]:
# ---------------- 1) One-way ----------------
n_per_group = 20
diets = np.repeat(["A","B","C"], n_per_group)
means = {"A":100, "B":105, "C":112}
y = np.concatenate([
    rng.normal(means["A"], 5, n_per_group),
    rng.normal(means["B"], 5, n_per_group),
    rng.normal(means["C"], 5, n_per_group),
])
df1 = pd.DataFrame({"Diet": diets, "Response": y})
m1 = smf.ols("Response ~ C(Diet)", data=df1).fit()
a1 = anova_lm(m1, typ=2)
explain(
    "1) ANOVA de una vía",
    "¿Las medias difieren entre niveles de Diet?",
    ["Factor: Diet ∈ {A,B,C}", "Respuesta: Response", "n=20 por grupo"],
    "Response ~ C(Diet)",
    ["p(C(Diet)) < 0.05 ⇒ al menos una media difiere; use Tukey para comparar pares."]
)

print("ANOVA 1-vía (Diet)", a1.round(4))


1) ANOVA de una vía
Pregunta que responde:
  - ¿Las medias difieren entre niveles de Diet?
Estructura de los datos:
  - Factor: Diet ∈ {A,B,C}
  - Respuesta: Response
  - n=20 por grupo
Fórmula del modelo:
  Response ~ C(Diet)
Cómo interpretar (mini guía):
  - p(C(Diet)) < 0.05 ⇒ al menos una media difiere; use Tukey para comparar pares.
ANOVA 1-vía (Diet)              sum_sq    df        F  PR(>F)
C(Diet)   1396.8231   2.0  36.6412     0.0
Residual  1086.4674  57.0      NaN     NaN


## Interpretación de los resultados

### Resumen de salida

| Fuente   | Suma de cuadrados (sum_sq) | gl | F     | p-valor |
| -------- | -------------------------- | -- | ----- | ------- |
| C(Diet)  | 1396.8231                  | 2  | 36.64 | 0.000   |
| Residual | 1086.4674                  | 57 | —     | —       |

---

### Interpretación paso a paso

1. **C(Diet)** representa el efecto del factor “Dieta” con 3 niveles (A, B, C).

   * (F = 36.64) es el estadístico de prueba calculado como:
     [
     F = \frac{\text{MS}*{\text{entre grupos}}}{\text{MS}*{\text{dentro de grupos}}}
     ]
   * Cuanto mayor es el valor de (F), mayor evidencia de diferencias entre medias.

2. **p-valor = 0.000** (en realidad < 0.001):

   * Como (p < 0.05), se **rechaza la hipótesis nula** (H_0).
   * Concluimos que **existen diferencias significativas entre las medias** de los grupos de dieta.

3. **Conclusión**:
   Al menos una de las dietas (A, B o C) produce una respuesta diferente en la variable *Response* (por ejemplo, glucosa, rendimiento, peso, etc.).
   No sabemos aún cuál difiere; para eso se aplican pruebas post-hoc, como **Tukey HSD**, que comparan los pares de medias.

---

### Ejemplo de interpretación narrativa

> El análisis de varianza de una vía mostró un efecto significativo de la dieta sobre la variable de respuesta ((F(2,57) = 36.64, p < 0.001)).
> Esto indica que las medias difieren entre al menos dos de las dietas evaluadas.
> Se recomienda realizar comparaciones post-hoc (por ejemplo, prueba de Tukey) para identificar específicamente entre qué grupos se encuentran dichas diferencias.

In [8]:
# ---------------- 2) Two-way + interaction ----------------
levels_diet = ["A","B","C"]
levels_sex = ["M","F"]
n_cell = 12
rows = []
for d in levels_diet:
    for s in levels_sex:
        mu = 100 + {"A":0,"B":5,"C":10}[d] + {"M":0,"F":2}[s]
        if d=="C" and s=="F":
            mu += 3
        rows += [(d,s,v) for v in rng.normal(mu,5,n_cell)]
df2 = pd.DataFrame(rows, columns=["Diet","Sex","Response"])
m2 = smf.ols("Response ~ C(Diet)*C(Sex)", data=df2).fit()
a2 = anova_lm(m2, typ=2)
explain(
    "2) ANOVA de dos vías con interacción",
    "¿Hay efectos de Diet, Sex y/o interacción Diet×Sex?",
    ["Factores: Diet {A,B,C}, Sex {M,F}", "Respuesta: Response", "Diseño factorial 3×2"],
    "Response ~ C(Diet)*C(Sex)",
    ["Revisa p(Diet), p(Sex) y p(Diet:Sex)."]
)
print("ANOVA 2-vías (Diet, Sex, Interacción)", a2.round(4))



2) ANOVA de dos vías con interacción
Pregunta que responde:
  - ¿Hay efectos de Diet, Sex y/o interacción Diet×Sex?
Estructura de los datos:
  - Factores: Diet {A,B,C}, Sex {M,F}
  - Respuesta: Response
  - Diseño factorial 3×2
Fórmula del modelo:
  Response ~ C(Diet)*C(Sex)
Cómo interpretar (mini guía):
  - Revisa p(Diet), p(Sex) y p(Diet:Sex).
ANOVA 2-vías (Diet, Sex, Interacción)                    sum_sq    df        F  PR(>F)
C(Diet)         1961.4396   2.0  44.0025  0.0000
C(Sex)           201.7384   1.0   9.0515  0.0037
C(Diet):C(Sex)   105.8845   2.0   2.3754  0.1009
Residual        1470.9966  66.0      NaN     NaN


In [9]:
# ---------------- 3) Three-way factorial ----------------
levels_diet3 = ["A","B"]
levels_sex3 = ["M","F"]
levels_age = ["Young","Old"]
n_cell3 = 10
rows3 = []
for d in levels_diet3:
    for s in levels_sex3:
        for a in levels_age:
            mu = 100 + {"A":0,"B":6}[d] + {"M":0,"F":2}[s] + {"Young":0,"Old":4}[a]
            if d=="B" and a=="Old":
                mu += 2
            rows3 += [(d,s,a,v) for v in rng.normal(mu,5,n_cell3)]
df3 = pd.DataFrame(rows3, columns=["Diet","Sex","AgeGroup","Response"])
m3 = smf.ols("Response ~ C(Diet)*C(Sex)*C(AgeGroup)", data=df3).fit()
a3 = anova_lm(m3, typ=2)
explain(
    "3) ANOVA factorial de 3 vías",
    "¿Qué efectos principales e interacciones (dobles y triple) explican la variación?",
    ["Factores: Diet {A,B}, Sex {M,F}, AgeGroup {Young,Old}", "Respuesta: Response"],
    "Response ~ C(Diet)*C(Sex)*C(AgeGroup)",
    ["Si hay interacción triple significativa, interpreta primero esa y luego las dobles."]
)

print("ANOVA 3-vías (efectos e interacciones)", a3.round(4))



3) ANOVA factorial de 3 vías
Pregunta que responde:
  - ¿Qué efectos principales e interacciones (dobles y triple) explican la variación?
Estructura de los datos:
  - Factores: Diet {A,B}, Sex {M,F}, AgeGroup {Young,Old}
  - Respuesta: Response
Fórmula del modelo:
  Response ~ C(Diet)*C(Sex)*C(AgeGroup)
Cómo interpretar (mini guía):
  - Si hay interacción triple significativa, interpreta primero esa y luego las dobles.
ANOVA 3-vías (efectos e interacciones)                                sum_sq    df        F  PR(>F)
C(Diet)                      811.5886   1.0  26.4728  0.0000
C(Sex)                       131.8373   1.0   4.3003  0.0417
C(AgeGroup)                  475.4192   1.0  15.5075  0.0002
C(Diet):C(Sex)                58.1144   1.0   1.8956  0.1728
C(Diet):C(AgeGroup)           78.8339   1.0   2.5714  0.1132
C(Sex):C(AgeGroup)             0.0158   1.0   0.0005  0.9820
C(Diet):C(Sex):C(AgeGroup)    43.8354   1.0   1.4298  0.2357
Residual                    2207.3325  72.0      

In [10]:
# ---------------- 4) Repeated Measures ----------------
n_subj = 24
times = ["Pre","Post","FollowUp"]
rows_rm = []
for subj in range(n_subj):
    intercept = rng.normal(100,6)
    eff = {"Pre":0, "Post":5, "FollowUp":3}
    for t in times:
        rows_rm.append((subj, t, rng.normal(intercept+eff[t],4)))
df4 = pd.DataFrame(rows_rm, columns=["Subject","Time","Response"])
rm = AnovaRM(df4, depvar="Response", subject="Subject", within=["Time"]).fit()
explain(
    "4) Medidas repetidas (intra-sujeto)",
    "¿La media cambia a través del tiempo en los mismos sujetos?",
    ["Within: Time {Pre,Post,FollowUp}", "Bloque aleatorio: Subject", "Respuesta: Response"],
    "AnovaRM(depvar='Response', subject='Subject', within=['Time'])",
    ["p(Time) < 0.05 ⇒ hay diferencia entre al menos dos tiempos."]
)
rm_table = pd.DataFrame({
    "Effect": rm.anova_table.index,
    "F Value": rm.anova_table["F Value"].round(4),
    "Num DF": rm.anova_table["Num DF"].round(4),
    "Den DF": rm.anova_table["Den DF"].round(4),
    "Pr > F": rm.anova_table["Pr > F"].round(5),
})
print("Medidas repetidas (Time dentro de sujeto)", rm_table)



4) Medidas repetidas (intra-sujeto)
Pregunta que responde:
  - ¿La media cambia a través del tiempo en los mismos sujetos?
Estructura de los datos:
  - Within: Time {Pre,Post,FollowUp}
  - Bloque aleatorio: Subject
  - Respuesta: Response
Fórmula del modelo:
  AnovaRM(depvar='Response', subject='Subject', within=['Time'])
Cómo interpretar (mini guía):
  - p(Time) < 0.05 ⇒ hay diferencia entre al menos dos tiempos.
Medidas repetidas (Time dentro de sujeto)      Effect  F Value  Num DF  Den DF   Pr > F
Time   Time   12.638     2.0    46.0  0.00004


In [11]:
# ---------------- 5) MANOVA ----------------
n_per_group_m = 25
diet_m = np.repeat(["A","B","C"], n_per_group_m)
mu_map = {"A":(100,50), "B":(105,48), "C":(112,55)}
Y1 = np.concatenate([rng.normal(mu_map[d][0],6,n_per_group_m) for d in ["A","B","C"]])
Y2 = np.concatenate([rng.normal(mu_map[d][1],5,n_per_group_m) for d in ["A","B","C"]])
df5 = pd.DataFrame({"Diet":diet_m, "Y1":Y1, "Y2":Y2})
man = MANOVA.from_formula("Y1 + Y2 ~ C(Diet)", data=df5)
mv = man.mv_test()
explain(
    "5) MANOVA",
    "¿Difieren simultáneamente Y1 y Y2 entre niveles de Diet?",
    ["Factor: Diet {A,B,C}", "Respuestas: Y1, Y2"],
    "Y1 + Y2 ~ C(Diet)",
    ["Revisa Pillai/Wilks y su p-valor; si es significativo, procede a análisis univariados y efectos."]
)
# univariados como apoyo
a5_y1 = anova_lm(smf.ols("Y1 ~ C(Diet)", data=df5).fit(), typ=2).round(4)
a5_y2 = anova_lm(smf.ols("Y2 ~ C(Diet)", data=df5).fit(), typ=2).round(4)
print("MANOVA apoyo: ANOVA univariado de Y1 y Y2", pd.concat({"Y1":a5_y1, "Y2":a5_y2}))
print("\nResumen MANOVA (extracto):")
print(mv.summary())


5) MANOVA
Pregunta que responde:
  - ¿Difieren simultáneamente Y1 y Y2 entre niveles de Diet?
Estructura de los datos:
  - Factor: Diet {A,B,C}
  - Respuestas: Y1, Y2
Fórmula del modelo:
  Y1 + Y2 ~ C(Diet)
Cómo interpretar (mini guía):
  - Revisa Pillai/Wilks y su p-valor; si es significativo, procede a análisis univariados y efectos.
MANOVA apoyo: ANOVA univariado de Y1 y Y2                 sum_sq    df        F  PR(>F)
Y1 C(Diet)   1562.3610   2.0  19.5257     0.0
   Residual  2880.5597  72.0      NaN     NaN
Y2 C(Diet)    438.3180   2.0  12.0758     0.0
   Residual  1306.7039  72.0      NaN     NaN

Resumen MANOVA (extracto):
                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept         Value   Num DF  Den DF  F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda   0.0079 2.0000 71.0000 4469.367

In [12]:
# ---------------- 6) ANCOVA ----------------
n = 80
treatment = rng.choice(["Control","Treated"], size=n)
age = rng.integers(25,65,size=n)
y_anc = 50 + 0.6*age + (treatment=="Treated")*4 + rng.normal(0,5,size=n)
df6 = pd.DataFrame({"Treatment":treatment, "Age":age, "Response":y_anc})
m6 = smf.ols("Response ~ C(Treatment) + Age", data=df6).fit()
a6 = anova_lm(m6, typ=2)
explain(
    "6) ANCOVA",
    "¿Hay efecto del tratamiento ajustando por Age?",
    ["Factor: Treatment {Control,Treated}", "Covariable: Age (continua)", "Respuesta: Response"],
    "Response ~ C(Treatment) + Age",
    ["Si p(C(Treatment)) < 0.05 ⇒ efecto tras ajustar por Age;",
     "Si p(Age) < 0.05 ⇒ Age está asociado con Response."]
)
print("ANCOVA (Treatment + Age)", a6.round(4))



6) ANCOVA
Pregunta que responde:
  - ¿Hay efecto del tratamiento ajustando por Age?
Estructura de los datos:
  - Factor: Treatment {Control,Treated}
  - Covariable: Age (continua)
  - Respuesta: Response
Fórmula del modelo:
  Response ~ C(Treatment) + Age
Cómo interpretar (mini guía):
  - Si p(C(Treatment)) < 0.05 ⇒ efecto tras ajustar por Age;
  - Si p(Age) < 0.05 ⇒ Age está asociado con Response.
ANCOVA (Treatment + Age)                  sum_sq    df         F  PR(>F)
C(Treatment)   171.2115   1.0    6.6896  0.0116
Age           3655.5178   1.0  142.8299  0.0000
Residual      1970.7002  77.0       NaN     NaN


In [13]:
# ---------------- 7) Mixed ANOVA via Linear Mixed Model ----------------
n_subj_mix = 40
groups = rng.choice(["Placebo","Drug"], size=n_subj_mix)
rows_mx = []
for i,g in enumerate(groups):
    subj = f"S{i:02d}"
    intercept = rng.normal(100,6)
    for t in ["Pre","Post"]:
        mu = intercept + (2 if (t=="Post" and g=="Placebo") else 0) + (8 if (t=="Post" and g=="Drug") else 0)
        rows_mx.append((subj,g,t,rng.normal(mu,4)))
df7 = pd.DataFrame(rows_mx, columns=["Subject","Group","Time","Response"])
mix = smf.mixedlm("Response ~ C(Group)*C(Time)", data=df7, groups="Subject").fit(reml=False)

explain(
    "7) Diseño mixto con efectos aleatorios",
    "¿El cambio en el tiempo difiere entre grupos, considerando la variabilidad entre sujetos?",
    ["Entre: Group {Placebo,Drug}", "Dentro: Time {Pre,Post}", "Aleatorio: intercepto por Subject"],
    "mixedlm('Response ~ C(Group)*C(Time)', groups='Subject')",
    ["Mira p(C(Group):C(Time)): si < 0.05, la mejora Post depende del grupo."]
)

# Construir tabla de coeficientes (robusta a cambios de versión)
coef_df = pd.DataFrame({
    "Term": mix.params.index,
    "Coef.": mix.params.values,
    "Std.Err.": mix.bse.values,
    "z": mix.tvalues.values,
    "P>|z|": mix.pvalues.values,
})
conf = mix.conf_int()
coef_df["[0.025"] = conf[0].values
coef_df["0.975]"] = conf[1].values
print("Modelo Mixto (coeficientes fijos)", coef_df.round(4))

print("\nResumen del modelo mixto (abreviado):")
print(mix.summary())



7) Diseño mixto con efectos aleatorios
Pregunta que responde:
  - ¿El cambio en el tiempo difiere entre grupos, considerando la variabilidad entre sujetos?
Estructura de los datos:
  - Entre: Group {Placebo,Drug}
  - Dentro: Time {Pre,Post}
  - Aleatorio: intercepto por Subject
Fórmula del modelo:
  mixedlm('Response ~ C(Group)*C(Time)', groups='Subject')
Cómo interpretar (mini guía):
  - Mira p(C(Group):C(Time)): si < 0.05, la mejora Post depende del grupo.
Modelo Mixto (coeficientes fijos)                                  Term     Coef.  Std.Err.        z   P>|z|  \
0                           Intercept  108.1168    1.5056  71.8080  0.0000   
1                 C(Group)[T.Placebo]   -7.4172    2.2445  -3.3046  0.0010   
2                      C(Time)[T.Pre]   -7.9229    1.1019  -7.1901  0.0000   
3  C(Group)[T.Placebo]:C(Time)[T.Pre]    8.0441    1.6427   4.8970  0.0000   
4                         Subject Var    2.7339    1.0227   2.6734  0.0075   

     [0.025    0.975]  
0  105.16